In [2]:
import numpy as np
import pandas as pd
np.random.seed(0)
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,RepeatVector,BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.initializers import glorot_uniform
import tensorflow as tf
from keras import backend as K
np.random.seed(1)

#**Fetching Data**

In [ ]:
data = pd.read_json("/content/drive/MyDrive/my projects/Github_bug_prediction/data/archive/embold_train.json")
test_data = pd.read_json("/content/drive/MyDrive/my projects/Github_bug_prediction/data/archive/embold_test.json")

In [ ]:
''' To drop row with NAN value'''
data=data.dropna()
test_data = test_data.dropna()

In [ ]:
data.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


In [ ]:
data.tail()

,title,body,label
149995,suggestion getinventorylist should return ra...,<!-- note: anything within these brackets will...,1
149996,decoder displays some neurovault images incorr...,"as noted by @nicholst, some neurovault images ...",0
149997,"parser should return an error, not an exception",for this raml:\r \r yaml\r %raml 1.0\r tit...,0
149998,errorexception array to string conversion on ...,see above.\r \r occurs in branch routes .,0
149999,ignore headings in code sections,do not process headings inside quotes and code...,0


In [ ]:
test_data.head()

,title,body
0,config question path-specific environment var...,issue description or question\r \r hey @artemg...
1,crash indien vol,de simulator crasht als hij vol zit
2,unable to mine rocks,"sarkasmo starting today, when i hit enter act..."
3,not all whitelists are processed,create following rules... order of creation is...
4,add ctx menu for idafree 70 and idafree 5,"associated with .dll, .dll_, .exe, .exe_, .sc,..."


In [ ]:
test_data.tail()

,title,body
29995,closing a project with running simulations doe...,should the client take care of that or should ...
29996,missing table description on db2 / db2 iseries...,"on tables, fields descriptions and tooltip are..."
29997,cases when notification should be shown,- new message - always but not on chat screen\...
29998,ip-2914. errors in idea2client when syncing,i see the following errors in the logs:\r \r ...
29999,portable open system pharmacology version,"hello,\r first of all, i would like to thank t..."


In [ ]:
X_train = list((data['title'].map(str) +' '+ data['body']).apply(lambda row: row.strip()))

X_test = list((test_data['title'].map(str) +' '+ test_data['body']).apply(lambda row: row.strip()))

In [ ]:
Y_train = list(data['label'])

#**preprocessing the data**

In [ ]:
import nltk
nltk.download('punkt')
import string
nltk.download('wordnet')
lemmer = nltk.stem.WordNetLemmatizer()
''' Stop words are not applied because in this dataset if we remove the stopwords it may cahne the meaning of the word'''
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


' Stop words are not applied because in this dataset if we remove the stopwords it may cahne the meaning of the word'

In [ ]:
def LemTokens(tokens):
    return list(map(lemmer.lemmatize,tokens))
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
def clean_data(X):
  return list(map(LemNormalize,X))

In [ ]:
X_train = clean_data(X_train) 
X_test = clean_data(X_test)

In [ ]:
print(X_train[0])

['yzoom', 'piano', 'roll', 'a', 'yzoom', 'on', 'the', 'piano', 'roll', 'would', 'be', 'useful']


Save the data

In [ ]:
import pickle
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/train.pkl","wb") as f:
  pickle.dump(X_train,f)
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/test.pkl","wb") as f:
  pickle.dump(X_test,f)
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/label.pkl","wb") as f:
  pickle.dump(Y_train,f)

#**Load the saved data**

In [ ]:
import pickle
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/train.pkl","rb") as f:
  X_train=pickle.load(f)
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/test.pkl","rb") as f:
  X_test=pickle.load(f)

In [11]:
import pickle
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/label.pkl","rb") as f:
  Y_train=pickle.load(f)

In [ ]:
print(X_train[0],X_test[0])

NameError: ignored

In [ ]:
print(Y_train[2])

1


#**Build the model**

Idea of this bert layer is taken from https://www.analyticsvidhya.com/blog/2020/10/simple-text-multi-classification-task-using-keras-bert/


importing bert layer from tensorflow hub

In [ ]:
!pip install bert-for-tf2 >> /dev/null

In [ ]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer
import os

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2021-04-20 08:15:35--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.208, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   236MB/s    in 1.6s    

2021-04-20 08:15:37 (236 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [ ]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
os.makedirs("model", exist_ok=True)

In [ ]:
!mv uncased_L-12_H-768_A-12/ model

In [ ]:
bert_model_name="uncased_L-12_H-768_A-12"

bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

In [ ]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

In [ ]:
a=tokenizer.tokenize("i am zcool")
print(a)
print(tokenizer.convert_tokens_to_ids(a))

['i', 'am', 'z', '##co', '##ol']
[1045, 2572, 1062, 3597, 4747]


In [3]:
import tensorflow_hub as hub
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=False)

In [4]:

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [5]:
def build_model(bert_layer,max_len=512):

  input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
  input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
  segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
  
  pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
  clf_output = sequence_output[:, 0, :]
  X = Dense(60,activation='relu',kernel_initializer=glorot_uniform(),kernel_regularizer=l2(0.01))(clf_output)
  X = BatchNormalization()(X)
  X = Dropout(rate=0.2)(X)

  X = Dense(30,activation='relu',kernel_initializer=glorot_uniform(),kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Dropout(rate=0.2)(X)

  X = Dense(10,activation='relu',kernel_initializer=glorot_uniform(),kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Dropout(rate=0.2)(X)

  output = Dense(3,activation='softmax')(X)

  model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=output)
  return model

In [6]:
model = build_model(bert_layer,512)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [8]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#**Training the model**

In [ ]:
X_train=list(map(" ".join,X_train))

In [ ]:
X_train_input = bert_encode(X_train,tokenizer,512)

save the encoded data

In [ ]:
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/input","wb") as f:
  pickle.dump(X_train_input,f)

load the encoded data

In [12]:
with open("/content/drive/MyDrive/my projects/Github_bug_prediction/saved_list/input","rb") as f:
  X_train=pickle.load(f)

In [13]:
id,mask,segment = X_train

changing the datatype to int 32

In [14]:
id=id.astype(np.int32)
mask=mask.astype(np.int32)
segment=segment.astype(np.int32)

In [15]:
Y_train = np.array(Y_train).reshape(150000,1)

In [16]:
print(Y_train.shape)

(150000, 1)


In [ ]:
model.fit([id,mask,segment],Y_train,epochs=3,batch_size=20)

Epoch 1/3
   1/7500 [..............................] - ETA: 117:29:27 - loss: 3.0935 - accuracy: 0.5000